In [1]:
!pip install stanza
import stanza
import sqlite3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 41.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
con = sqlite3.connect("proyectoPura.db")
cur = con.cursor()

query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""

cur.execute(query)
titles = cur.fetchall()
#print(titles)

titles_list = []
for title in titles:
  if title[0] is not None:
    titles_list.append(title[0])
  if title[0] is None:
    titles_list.append("NULL") #we take into account "NULL" titles, to keep the same structure as original DB.
print(titles_list)

con.close()

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Al

In [5]:
#Delimitation of titles (later recognizing them)
import re
'''def find_delimit_subtit(text):
  #subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r' | ', text)
  subtitle_ = re.sub(r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*', r'" | ', text)## N: He restituído la comilla de cierre que te habías perdido, igual es problemático para los que no ponían comillas... lo vamos viendo
  return subtitle_'''

def subtitle_format(matchobj):
  match = matchobj.group(0)
  if re.fullmatch(r'\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*-\s*"\s*', match):
    return ' | "'
  elif re.fullmatch(r'\s*-\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*\[\.\.\.\]\s*"\s*-\s*', match):
    submatch = re.fullmatch(r'(\s*)\[\.\.\.\]\s*"\s*-\s*', match)
    return submatch.group(1)+'[...]" | '

def formatting_subtitles(text):
    subtit_re = r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*'
    new_text = re.sub(subtit_re, subtitle_format, text)
    return new_text

encontrar_subtit = [formatting_subtitles(row) for row in titles_list]
list_titsub = [row for row in encontrar_subtit if " | " in row]

#only this exceptions do not adjust to pattern (2 rows):
excepcion_1 = """Piquer-Oda "Alzad vuestras"""
excepcion_2 = """Matrimonios por anuncios-Los baños de Biarritz"""
def subadhoc(matchobj):
  return ' "'+ matchobj.group(0)[1:]
for i, row in enumerate(encontrar_subtit):
  if excepcion_1 in row:
    encontrar_subtit[i] = row.replace('Piquer-Oda "Alzad', 'Piquer-Oda. "Alzad') #Mismo formato que algunos sonetos.
  if excepcion_2 in row:
    encontrar_subtit[i]= row.replace('-', ' | ')
    #encontrar_subtit[i] = re.sub(r'[^|]+\]"', subadhoc, encontrar_subtit[i])##N: añadido mío (con la definición de subadhoc)

print(encontrar_subtit[249])
print(encontrar_subtit[207])

Inauguración del teatro de D. José Piquer-Oda. "Alzad vuestras tumbas [...]"
Matrimonios por anuncios | Los baños de Biarritz | "Hace pocos días [...]"


In [8]:
#we continue preparing each title to process it through model (tok,lemm,pos,depparse,ner)
import re

#def clean_per_title(title):
  #cleaned_text = re.sub(
    #r'\s*\(\s*(Continuación|Fragmentos?|Conclusión|Conclusiones?|Conclusion)\.?\s*\)\s*', #not now, mayber after processing
    #' ',
    #title)
  #cleaned_text = re.sub(r'\s*"\s*|\s*\[\s*\.\s*\.\s*\.\s*\]\s*|\s*\[\s*\u2026\s*\]\s*', ' ', cleaned_text)
  #return cleaned_text.strip()

#cleaned_rows = [clean_per_title(row) for row in encontrar_subtit]

#print(cleaned_rows)

cleaned_rows = [row.split(" | ") if " | " in row else [row] for row in encontrar_subtit]

for row in cleaned_rows:
  print(row, "\n")

['Al Sr. D. R. P. Enviándole mis obras'] 

['La mujer piadosa (Fragmentos)'] 

['Desvarío'] 

['Soneto. "¡Gloria y pobreza! El inmortal Quintana..."'] 

['Balada. La despedida de una hija'] 

['Un viaje a la Luna. Prólogo: La pesadilla de Mocquet'] 

['En el álbum de la señora M…'] 

['La primavera'] 

['La perejilera'] 

['Un viaje a la Luna. (Continuación)'] 

['Al señor Don B. M.'] 

['A orillas del Manzanares. La súplica'] 

['Un viaje a la Luna. (Continuación)'] 

['Fragmentos de la Rosa de Alejandra'] 

['Gente morena'] 

['Un viaje a la Luna. (Continuación)'] 

['Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo'] 

['A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños'] 

['Costumbres del siglo XIX. I'] 

['Margarita. Novela bretona'] 

['Los escondidos y la tapada. Fragmentos'] 

['A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum'] 

['Costumbres del siglo XIX.II'] 

['Margarita. Novela bretona. (Continuación)'] 

In [10]:
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text != "NULL":
    return [(word.text, word.lemma, word.upos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]
  else:
    return [("NULL")]
nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_titles_es = [[processing_1(el, nlp_es) for el in lista] for lista in cleaned_rows]

for element in processed_titles_es:
  print(element, "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('A', 'a', 'ADP', 3, 'case'), ('el', 'el', 'DET', 3, 'det'), ('Sr.', 'sr.', 'PROPN', 7, 'obl'), ('D.', 'd.', 'PROPN', 3, 'flat'), ('R.', 'R.', 'PROPN', 3, 'flat'), ('P.', 'p.', 'PROPN', 3, 'flat'), ('Enviando', 'enviar', 'VERB', 0, 'root'), ('le', 'él', 'PRON', 7, 'obl:arg'), ('mis', 'mi', 'DET', 10, 'det'), ('obras', 'obra', 'NOUN', 7, 'obj')]] 

[[('La', 'el', 'DET', 2, 'det'), ('mujer', 'mujer', 'NOUN', 0, 'root'), ('piadosa', 'piadoso', 'ADJ', 2, 'amod'), ('(', '(', 'PUNCT', 5, 'punct'), ('Fragmentos', 'Fragmentos', 'PROPN', 2, 'appos'), (')', ')', 'PUNCT', 5, 'punct')]] 

[[('Desvarío', 'desvarío', 'NOUN', 0, 'root')]] 

[[('Soneto', 'soneto', 'PROPN', 0, 'root'), ('.', '.', 'PUNCT', 1, 'punct'), ('"', '"', 'PUNCT', 3, 'punct'), ('¡', '¡', 'PUNCT', 3, 'punct'), ('Gloria', 'gloria', 'PROPN', 0, 'root'), ('y', 'y', 'CCONJ', 5, 'cc'), ('pobreza', 'pobreza', 'NOUN', 3, 'conj'), ('!', '!', 'PUNCT', 3, 'punct'), ('El', 'el', 'DET', 3, 'det'), ('inmortal', 'inmortal', 'ADJ', 3, 'amod')